In [48]:
import tkinter as tk
from tkinter import filedialog
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import os

def read_skeleton_data(filename):
    data = {}
    with open(filename, 'r') as file:
        for line in file:
            values = line.strip().split(',')
            if len(values) < 4:
                print(f"Skipping malformed line: {line.strip()}")
                continue
            try:
                frame, joint, x, y = map(int, values[:4])
                if frame not in data:
                    data[frame] = {}
                data[frame][joint] = (x, y)
            except ValueError as e:
                print(f"Error parsing line: {line.strip()} - {e}")
                continue
    return data

def calculate_angles_and_length(data):
    frames = sorted(data.keys())
    left_knee_angles, left_hip_angles, body_lengths = [], [], []

    for frame in frames:
        joints = data[frame]
        if all(k in joints for k in [12, 14, 16, 6, 10]):
            left_knee_angles.append(calculate_angle(joints[12], joints[14], joints[16]))
            left_hip_angles.append(calculate_angle(joints[6], joints[12], joints[16]))
            body_lengths.append(calculate_distance(joints[6], joints[12]))

    return frames, left_knee_angles, left_hip_angles, body_lengths

def calculate_angle(p1, p2, p3):
    a, b, c = np.array(p1), np.array(p2), np.array(p3)
    ba, bc = a - b, c - b

    norm_ba = np.linalg.norm(ba)
    norm_bc = np.linalg.norm(bc)

    if norm_ba == 0 or norm_bc == 0:
        return 0

    cosine_angle = np.dot(ba, bc) / (norm_ba * norm_bc)
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    return np.degrees(np.arccos(cosine_angle))

def calculate_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))


def read_barbell_positions(filename):
    frames, x_coords, y_coords = [], [], []
    with open(filename, 'r') as file:
        for line in file:
            frame, x, y = map(float, line.strip().split(',')[:3])
            frames.append(int(frame))
            x_coords.append(x)
            y_coords.append(y)
    return frames, x_coords, y_coords

def save_plot(fig):
    folder_selected = filedialog.askdirectory(title="Select Folder to Save Plot")
    if folder_selected:
        file_path = os.path.join(folder_selected, "combined_plot.png")
        fig.savefig(file_path)
        print(f"Plot saved to {file_path}")

def on_click(event):
    ax.plot(event.xdata, event.ydata, 'ro')
    canvas.draw()

def motion(event):
    if event.xdata is not None and event.ydata is not None:
        root.title(f"Skeleton and Barbell Metrics - Mouse at ({event.xdata:.2f}, {event.ydata:.2f})")

def plot_metrics_in_tkinter():
    global root, canvas, ax
    root = tk.Tk()
    root.title("Skeleton and Barbell Metrics")
    root.geometry("800x800")

    fig = Figure(figsize=(8, 6), dpi=100)
    ax = fig.add_subplot(1, 1, 1)

    ax.plot(skeleton_frames, left_knee_angles, label='Left Knee Angle', color='blue')
    ax.plot(skeleton_frames, left_hip_angles, label='Left Hip Angle', color='green')
    ax.plot(skeleton_frames, body_lengths, label='Body Length', color='red')
    ax.plot(barbell_frames, x_coords, label='Barbell X Position', color='purple')
    ax.plot(barbell_frames, y_coords, label='Barbell Y Position', color='orange')

    ax.set_title("Combined Metrics Over Time")
    ax.set_xlabel("Frame")
    ax.set_ylabel("Value")
    ax.legend()
    ax.grid()

    canvas = FigureCanvasTkAgg(fig, root)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    canvas.mpl_connect("button_press_event", on_click)
    canvas.mpl_connect("motion_notify_event", motion)

    save_button = tk.Button(root, text="Save Plot", command=lambda: save_plot(fig))
    save_button.pack(pady=10)

    root.mainloop()

# skeleton_file_path = 'D:\\labdata\\MOCAP\\recordings_copy\\受試者27\\recording_20241209_100647\\interpolated_mediapipe_landmarks_1.txt'
skeleton_file_path = 'D:\\labdata\\MOCAP\\recordings_copy\\受試者10\\recording_20241204_100047\\yolo_skeleton_coordinates_1st_interp.txt'
barbell_file_path = 'D:\\labdata\\MOCAP\\recordings_copy\\受試者10\\recording_20241204_100047\\yolo_coordinates_interpolated.txt'

skeleton_data = read_skeleton_data(skeleton_file_path)
skeleton_frames, left_knee_angles, left_hip_angles, body_lengths = calculate_angles_and_length(skeleton_data)
barbell_frames, x_coords, y_coords = read_barbell_positions(barbell_file_path)

plot_metrics_in_tkinter()


In [2]:
import tkinter as tk
from tkinter import filedialog
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import os
from scipy.signal import find_peaks, savgol_filter

def read_skeleton_data(filename):
    data = {}
    with open(filename, 'r') as file:
        for line in file:
            values = line.strip().split(',')
            if len(values) < 4:
                continue
            try:
                frame, joint, x, y = map(int, values[:4])
                if frame not in data:
                    data[frame] = {}
                data[frame][joint] = (x, y)
            except ValueError:
                continue
    return data

def calculate_angles(data):
    frames = sorted(data.keys())
    angles = []

    for frame in frames:
        joints = data[frame]
        if all(k in joints for k in [11, 13, 15]):
            angles.append(calculate_angle(joints[11], joints[13], joints[15]))
        else:
            angles.append(None)

    return frames, angles

def calculate_angle(p1, p2, p3):
    a, b, c = np.array(p1), np.array(p2), np.array(p3)
    ba, bc = a - b, c - b
    norm_ba = np.linalg.norm(ba)
    norm_bc = np.linalg.norm(bc)
    if norm_ba == 0 or norm_bc == 0:
        return 0
    cosine_angle = np.dot(ba, bc) / (norm_ba * norm_bc)
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    return np.degrees(np.arccos(cosine_angle))

def find_valleys(smoothed_angles, peaks, search_range=10):
    valleys = []
    valleys1 = []
    for peak in peaks:
        # 在峰值前後各 `search_range` 幀內找最小值
        left_bound = max(0, peak - search_range)
        right_bound = min(len(smoothed_angles) - 1, peak + search_range)

        left_min_index = left_bound + np.argmin(smoothed_angles[left_bound:peak])
        right_min_index = peak + np.argmin(smoothed_angles[peak:right_bound + 1])

        valleys.append(left_min_index)
        valleys1.append(right_min_index)

    return np.array(valleys),np.array(valleys1)

def plot_metrics_in_tkinter():
    global root, canvas, ax
    root = tk.Tk()
    root.title("Smoothed Left Knee Angle")
    root.geometry("800x600")

    fig = Figure(figsize=(8, 6), dpi=100)
    ax = fig.add_subplot(1, 1, 1)

    # 過濾掉 None 值
    valid_frames = [frames[i] for i in range(len(left_knee_angles)) if left_knee_angles[i] is not None]
    valid_angles = np.array([angle for angle in left_knee_angles if angle is not None])

    # **平滑化曲線**
    smoothed_angles = savgol_filter(valid_angles, window_length=15, polyorder=3)  # 避免過度震盪

    # 找出 170 度以上的峰值，間隔 10 幀以上
    peaks, _ = find_peaks(smoothed_angles, height=170, distance=10, prominence=5)

    # 找出峰值前後的谷底
    valleys,valleys1 = find_valleys(smoothed_angles, peaks, search_range=50)

    # 繪製平滑化曲線
    ax.plot(valid_frames, smoothed_angles, label='Smoothed Left Knee Angle', color='blue')
    ax.plot(np.array(valid_frames)[peaks], smoothed_angles[peaks], 'ro', label="Peaks")  # 紅色標記峰值
    ax.plot(np.array(valid_frames)[valleys], smoothed_angles[valleys], 'yo', label="Valleys")  # 藍色標記谷底
    ax.plot(np.array(valid_frames)[valleys1], smoothed_angles[valleys1], 'go', label="Valleys1")  # 藍色標記谷底


    ax.set_title("Smoothed Left Knee Angle with Peaks and Valleys")
    ax.set_xlabel("Frame")
    ax.set_ylabel("Angle (degrees)")
    ax.legend()
    ax.grid()

    canvas = FigureCanvasTkAgg(fig, root)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    root.mainloop()

# 設定你的檔案路徑
skeleton_file_path = 'D:\\labdata\\MOCAP\\recordings_copy\\受試者10\\recording_20241204_100047\\yolo_skeleton_coordinates_1st_interp.txt'

# 讀取數據
skeleton_data = read_skeleton_data(skeleton_file_path)
frames, left_knee_angles = calculate_angles(skeleton_data)

# 顯示圖表
plot_metrics_in_tkinter()


KeyboardInterrupt: 

In [1]:
import tkinter as tk
from tkinter import filedialog
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import os
from scipy.signal import find_peaks, savgol_filter
import cv2

def read_skeleton_data(filename):
    data = {}
    with open(filename, 'r') as file:
        for line in file:
            values = line.strip().split(',')
            if len(values) < 4:
                continue
            try:
                frame, joint, x, y = map(int, values[:4])
                if frame not in data:
                    data[frame] = {}
                data[frame][joint] = (x, y)
            except ValueError:
                continue
    return data

def calculate_angles(data):
    frames = sorted(data.keys())
    angles = []

    for frame in frames:
        joints = data[frame]
        if all(k in joints for k in [12, 14, 16]):
            angles.append(calculate_angle(joints[12], joints[14], joints[16]))
        else:
            angles.append(None)

    return frames, angles

def calculate_angle(p1, p2, p3):
    a, b, c = np.array(p1), np.array(p2), np.array(p3)
    ba, bc = a - b, c - b
    norm_ba = np.linalg.norm(ba)
    norm_bc = np.linalg.norm(bc)
    if norm_ba == 0 or norm_bc == 0:
        return 0
    cosine_angle = np.dot(ba, bc) / (norm_ba * norm_bc)
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    return np.degrees(np.arccos(cosine_angle))

def find_valleys(smoothed_angles, peaks, search_range=10, min_valley_value=170, min_depth=10):
    valleys = []
    valleys1 = []

    for peak in peaks:
        # 在峰值前後各 `search_range` 幀內找最小值
        left_bound = max(0, peak - search_range)
        right_bound = min(len(smoothed_angles) - 1, peak + search_range)

        # 找到左邊和右邊的波谷
        left_min_index = left_bound + np.argmin(smoothed_angles[left_bound:peak])
        right_min_index = peak + np.argmin(smoothed_angles[peak:right_bound + 1])

        left_min_value = smoothed_angles[left_min_index]
        right_min_value = smoothed_angles[right_min_index]
        peak_value = smoothed_angles[peak]

        # 檢查條件：波谷值小於 170，且谷底夠深 (峰值 - 谷底 >= min_depth)
        if left_min_value < min_valley_value and (peak_value - left_min_value) >= min_depth:
            valleys.append(left_min_index)

        if right_min_value < min_valley_value and (peak_value - right_min_value) >= min_depth:
            valleys1.append(right_min_index)

    return valleys, valleys1

def read_yolo_data(yolo_file):
    """ 讀取 yolo_coordinates_interpolated.txt，回傳 {frame: x} 字典 """
    yolo_data = {}
    with open(yolo_file, 'r') as file:
        for line in file:
            values = line.strip().split(',')
            if len(values) < 3:
                continue
            try:
                frame = int(values[0])
                x_value = float(values[2])  # 第三個數值是 X 座標
                yolo_data[frame] = x_value
            except ValueError:
                continue
    return yolo_data

def split_video(video_path, output_folder, valleys, valleys1, yolo_file, fps=30):
    """
    根據 valleys (黃色谷底) 和 valleys1 (綠色谷底) 分割影片，
    **只存 X 座標變化超過 20 的影片**，如果變化 <= 20，則不存成影片。
    """
    print(123)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 讀取影片
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 格式

    # 讀取 yolo 數據
    yolo_data = read_yolo_data(yolo_file)

    for i, (start_frame, end_frame) in enumerate(zip(valleys, valleys1)):
        if start_frame >= end_frame or start_frame >= total_frames or end_frame >= total_frames:
            print(f"跳過無效範圍: 開始 {start_frame}, 結束 {end_frame}")
            continue

        # 取得該範圍內的 yolo X 值變化
        x_values = [yolo_data[f] for f in range(start_frame, end_frame + 1) if f in yolo_data]
        x_range = max(x_values) - min(x_values) if x_values else 0

        # **如果變化小於等於 20，則跳過該片段**
        if x_range <= 150 or np.isnan(x_range):
            print(f"跳過片段 {i+1}（X 變化範圍: {x_range:.2f}，過小）")
            continue

        # 存影片
        output_path = os.path.join(output_folder, f"clip_{i+1}.mp4")
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        for frame_idx in range(start_frame, end_frame + 1):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        out.release()
        print(f"影片儲存: {output_path}（X 變化範圍: {x_range:.2f}）")

    cap.release()
    print("所有片段處理完成！")


def split_skeleton_data(original_skeleton_path, output_folder, valleys, valleys1, yolo_file):
    """
    根據 valleys (黃色谷底) 和 valleys1 (綠色谷底) 分割骨架數據，
    **只存 X 座標變化超過 20 的片段**，如果變化 <= 20，則不存。
    """
    # 讀取骨架數據檔案
    with open(original_skeleton_path, 'r') as file:
        lines = file.readlines()

    # 讀取 yolo 數據
    yolo_data = read_yolo_data(yolo_file)

    for i, (start_frame, end_frame) in enumerate(zip(valleys, valleys1)):
        if start_frame >= end_frame:
            print(f"跳過無效範圍: 開始 {start_frame}, 結束 {end_frame}")
            continue

        # 計算該片段內的 X 座標變化範圍
        x_values = [yolo_data[f] for f in range(start_frame, end_frame + 1) if f in yolo_data]
        x_range = max(x_values) - min(x_values) if x_values else 0

        # **如果變化小於等於 20，則跳過該片段**
        if x_range <= 150 or np.isnan(x_range):
            print(f"跳過骨架數據 {i+1}（X 變化範圍: {x_range:.2f}，過小）")
            continue

        # 存骨架數據
        clip_folder = os.path.join(output_folder)
        os.makedirs(clip_folder, exist_ok=True)

        output_skeleton_path = os.path.join(clip_folder, f"skeleton_{i+1}.txt")
        with open(output_skeleton_path, 'w') as out_file:
            for line in lines:
                values = line.strip().split(',')
                if len(values) < 4:
                    continue
                try:
                    frame = int(values[0])  # 解析 frame 數
                    if start_frame <= frame <= end_frame:
                        out_file.write(line)  # 保留符合範圍的行
                except ValueError:
                    continue

        print(f"骨架數據儲存: {output_skeleton_path}（Y 變化範圍: {x_range:.2f}）")

    print("所有骨架數據處理完成！")


def split_bar_data(original_bar_path, output_folder, valleys, valleys1, yolo_file):
    """
    根據 valleys (黃色谷底) 和 valleys1 (綠色谷底) 分割槓鈴數據，
    **只存 X 座標變化超過 20 的片段**，如果變化 ≤ 20，則不存。
    """
    # 讀取槓鈴數據檔案
    with open(original_bar_path, 'r') as file:
        lines = file.readlines()

    # 讀取 yolo 數據
    yolo_data = read_yolo_data(yolo_file)

    for i, (start_frame, end_frame) in enumerate(zip(valleys, valleys1)):
        if start_frame >= end_frame:
            print(f"跳過無效範圍: 開始 {start_frame}, 結束 {end_frame}")
            continue

        # 計算該片段內的 X 座標變化範圍
        x_values = [yolo_data[f] for f in range(start_frame, end_frame + 1) if f in yolo_data]
        x_range = max(x_values) - min(x_values) if x_values else 0

        # **如果變化小於等於 20，則跳過該片段**
        if x_range <= 150 or np.isnan(x_range):
            print(f"跳過槓鈴數據 {i+1}（X 變化範圍: {x_range:.2f}，過小）")
            continue

        # 存槓鈴數據
        clip_folder = os.path.join(output_folder)
        os.makedirs(clip_folder, exist_ok=True)

        output_bar_path = os.path.join(clip_folder, f"bar_{i+1}.txt")
        with open(output_bar_path, 'w') as out_file:
            for line in lines:
                values = line.strip().split(',')
                if len(values) < 4:
                    continue
                try:
                    frame = int(values[0])  # 解析 frame 數
                    if start_frame <= frame <= end_frame:
                        out_file.write(line)  # 保留符合範圍的行
                except ValueError:
                    continue

        print(f"槓鈴數據儲存: {output_bar_path}（X 變化範圍: {x_range:.2f}）")

    print("所有槓鈴數據處理完成！")
    

        
def plot_metrics_in_tkinter(yolo_file):
    global root, canvas, valleys, valleys1

    # Tkinter UI 初始化
    root = tk.Tk()
    root.title("Left Knee Angle & Bar Y-Axis Analysis")
    root.geometry("1000x800")

    # 圖表初始化
    fig, axes = Figure(figsize=(10, 8), dpi=100), [None, None, None]
    axes[0] = fig.add_subplot(3, 1, 1)  # 原始角度圖
    axes[1] = fig.add_subplot(3, 1, 2)  # 平滑後角度圖
    axes[2] = fig.add_subplot(3, 1, 3)  # 槓鈴Y座標圖

    # 過濾角度資料中的 None
    valid_frames = [frames[i] for i in range(len(left_knee_angles)) if left_knee_angles[i] is not None]
    valid_angles = np.array([angle for angle in left_knee_angles if angle is not None])

    # 平滑化角度
    smoothed_angles = savgol_filter(valid_angles, window_length=11, polyorder=3)
    peaks, _ = find_peaks(smoothed_angles, height=160, distance=55, prominence=5)

    # 波谷檢測
    all_valleys, all_valleys1 = find_valleys(smoothed_angles, peaks, search_range=90, min_valley_value=170, min_depth=10)

    valleys, valleys1 = [], []
    last_start, last_end = -1, -1  # 初始化前一段範圍
    
    for peak in peaks:
        left_valley = next((v for v in reversed(all_valleys) if v < peak), None)
        right_valley = next((v for v in all_valleys1 if v > peak), None)
    
        if left_valley is not None and right_valley is not None:
            new_start, new_end = left_valley, right_valley
    
            # 檢查是否和上一段有超過 50% 重疊
            if last_start != -1 and last_end != -1:
                overlap_start = max(new_start, last_start)
                overlap_end = min(new_end, last_end)
                overlap = max(0, overlap_end - overlap_start)
                len_last = last_end - last_start
                len_new = new_end - new_start
    
                if overlap / min(len_last, len_new) > 0.5:
                    print(f"跳過重疊片段：({new_start}, {new_end}) 和上一段重疊 {overlap} 幀")
                    continue  # 跳過這個片段
    
            valleys.append(new_start)
            valleys1.append(new_end)
            last_start, last_end = new_start, new_end  # 更新上一段範圍


    


    # === [1] 原始角度圖 ===
    axes[0].plot(valid_frames, valid_angles, color='gray', alpha=0.7)
    axes[0].plot(np.array(valid_frames)[peaks], valid_angles[peaks], 'ro')
    axes[0].plot(np.array(valid_frames)[valleys], valid_angles[valleys], 'yo')
    axes[0].plot(np.array(valid_frames)[valleys1], valid_angles[valleys1], 'go')
    axes[0].set_title("Raw Left Knee Angle")
    axes[0].set_ylabel("Angle (deg)")
    axes[0].grid()

    # === [2] 平滑角度圖 ===
    axes[1].plot(valid_frames, smoothed_angles, color='blue')
    axes[1].plot(np.array(valid_frames)[peaks], smoothed_angles[peaks], 'ro')
    axes[1].plot(np.array(valid_frames)[valleys], smoothed_angles[valleys], 'yo')
    axes[1].plot(np.array(valid_frames)[valleys1], smoothed_angles[valleys1], 'go')
    axes[1].set_title("Smoothed Left Knee Angle")
    axes[1].set_ylabel("Angle (deg)")
    axes[1].grid()

    # === [3] 槓鈴Y座標變化圖 ===
    yolo_data = read_yolo_data(yolo_file)
    bar_y_frames = sorted(yolo_data.keys())
    bar_y_values = [yolo_data[frame] for frame in bar_y_frames]

    axes[2].plot(bar_y_frames, bar_y_values, color='purple', label='Bar Y-axis')
    axes[2].set_title("Barbell Y-Coordinate Over Time")
    axes[2].set_xlabel("Frame")
    axes[2].set_ylabel("Y Position")
    axes[2].grid()

    
    # 標記左右谷底區間
    # 僅標記被實際儲存的片段（X 變化範圍 > 50）
    for v_start, v_end in zip(valleys, valleys1):
        x_values = [yolo_data[f] for f in range(valid_frames[v_start], valid_frames[v_end] + 1) if f in yolo_data]
        x_range = max(x_values) - min(x_values) if x_values else 0
    
        if x_range > 150:
            axes[2].axvspan(valid_frames[v_start], valid_frames[v_end], color='yellow', alpha=0.2)




    # === 儲存圖像 ===



    print(valleys, valleys1)
    fig.savefig(path+"\\left_knee_angle_plot.png")  # 儲存為 PNG 圖檔
    print(path+"\\left_knee_angle_plot.png")
    canvas = FigureCanvasTkAgg(fig, root)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    # root.mainloop()

    
# path = "D:/labdata/MOCAP/recordings_copy/受試者11/recording_20241204_112324/"
# # 設定你的檔案路徑
# skeleton_file_path = os.path.join(path, f'yolo_skeleton_coordinates_1st_interp.txt')
# # 設定影片路徑
# video_path =  os.path.join(path, f'vision1.avi')
# output_folder = os.path.join(path, f'clips')
# output_folder1 = os.path.join(path, f'skeleton')
# # 讀取數據
# skeleton_data = read_skeleton_data(skeleton_file_path)
# frames, left_knee_angles = calculate_angles(skeleton_data)
# plot_metrics_in_tkinter()
# split_skeleton_data(skeleton_file_path,output_folder1,valleys,valleys1)

# for i in range(15):
#     base_path = f"D:/labdata/MOCAP/recordings_copy/受試者{56+i}"+"/"

base_path = "D:/labdata/MOCAP/recordings_copy/"
# 取得 base_path 下的所有子資料夾
for sub_folder in os.listdir(base_path):
    midpath = os.path.join(base_path, sub_folder)
    
    # 確保 path 是資料夾
    if not os.path.isdir(midpath):
        continue

    for sub_folder1 in os.listdir(midpath):
        path = os.path.join(midpath, sub_folder1)
        
        # 確保 path 是資料夾
        if not os.path.isdir(path):
            continue
    
    
        # 設定檔案路徑
        skeleton_file_path = os.path.join(path, 'yolo_skeleton_coordinates_vision1_interp.txt')
        bar_file_path = os.path.join(path, 'yolo_coordinates_interpolated.txt')
        video_path = os.path.join(path, 'vision1.avi')
        output_folder = os.path.join(path, 'clips')
        output_folder1 = os.path.join(path, 'skeleton_vision1')
    
        output_folder2 = os.path.join(path, 'bar')
    
        # 檢查檔案是否存在，避免錯誤
        if not os.path.exists(skeleton_file_path):
            print(f"檔案 {skeleton_file_path} 不存在，跳過該資料夾")
            continue
    
    #         讀取數據
        skeleton_data = read_skeleton_data(skeleton_file_pat-h)
        frames, left_knee_angles = calculate_angles(skeleton_data)
    
        # 顯示波形圖
        plot_metrics_in_tkinter(bar_file_path)
        # split_video(video_path, output_folder, valleys, valleys1,bar_file_path, fps=30)
        # # 執行切割
        split_skeleton_data(skeleton_file_path, output_folder1, valleys, valleys1,bar_file_path)
        # split_bar_data(bar_file_path, output_folder2, valleys, valleys1,bar_file_path)




檔案 D:/labdata/MOCAP/recordings_copy/受試者10\recording_20241204_094919棋盤\yolo_skeleton_coordinates_vision1_interp.txt 不存在，跳過該資料夾
[37, 108, 205, 279, 371, 443, 544, 623, 679, 753] [108, 205, 279, 371, 443, 544, 623, 679, 753, 908]
D:/labdata/MOCAP/recordings_copy/受試者10\recording_20241204_100047\left_knee_angle_plot.png
骨架數據儲存: D:/labdata/MOCAP/recordings_copy/受試者10\recording_20241204_100047\skeleton_vision1\skeleton_1.txt（Y 變化範圍: 309.94）
骨架數據儲存: D:/labdata/MOCAP/recordings_copy/受試者10\recording_20241204_100047\skeleton_vision1\skeleton_2.txt（Y 變化範圍: 266.07）
骨架數據儲存: D:/labdata/MOCAP/recordings_copy/受試者10\recording_20241204_100047\skeleton_vision1\skeleton_3.txt（Y 變化範圍: 287.26）
骨架數據儲存: D:/labdata/MOCAP/recordings_copy/受試者10\recording_20241204_100047\skeleton_vision1\skeleton_4.txt（Y 變化範圍: 261.01）
骨架數據儲存: D:/labdata/MOCAP/recordings_copy/受試者10\recording_20241204_100047\skeleton_vision1\skeleton_5.txt（Y 變化範圍: 254.28）
骨架數據儲存: D:/labdata/MOCAP/recordings_copy/受試者10\recording_20241204_100047\skele

In [2]:
import cv2

def main():
    # 讀取影片
    video_path = "D:\\labdata\\MOCAP\\recordings_copy\\受試者11\\recording_20241204_113115\\vision1.avi"  # 請將此處替換為影片的路徑
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("無法開啟影片")
        return

    paused = False
    frame_number = 0
    pause_points_1 = []  # 第一個矩陣
    pause_points_2 = []  # 第二個矩陣
    pause_count = 0  # 記錄目前暫停的次數

    def mouse_callback(event, x, y, flags, param):
        nonlocal paused, pause_count
        if event == cv2.EVENT_LBUTTONDOWN:
            paused = not paused
            if paused:
                print(f"暫停於幀數: {frame_number}")
                
#                 交替存入兩個矩陣
                if pause_count % 2 == 0:
                    pause_points_1.append(frame_number)
                else:
                    pause_points_2.append(frame_number)
#                 pause_points_1.append(frame_number)
                pause_count += 1  # 增加計數

    while True:
        if not paused:
            ret, frame = cap.read()
            if not ret:
                print("影片播放結束或讀取錯誤")
                break
            frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

        # 顯示影片畫面
        cv2.imshow("Video", frame)

        # 設定滑鼠事件
        cv2.setMouseCallback("Video", mouse_callback)

        # 等待鍵盤或滑鼠事件
        key = cv2.waitKey(30) & 0xFF
        
        if key == ord('q'):
            print("退出程式")
            break

    # 顯示儲存的暫停幀數
    print("第一個矩陣 (pause_points_1):", pause_points_1)
    print("第二個矩陣 (pause_points_2):", pause_points_2)

    # 釋放資源
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


暫停於幀數: 99
暫停於幀數: 202
暫停於幀數: 221
暫停於幀數: 332
暫停於幀數: 370
暫停於幀數: 492
暫停於幀數: 526
暫停於幀數: 635
暫停於幀數: 670
暫停於幀數: 768
暫停於幀數: 801
暫停於幀數: 896
暫停於幀數: 918
暫停於幀數: 1051
暫停於幀數: 1084
暫停於幀數: 1194
暫停於幀數: 1223
暫停於幀數: 1329
暫停於幀數: 1354
暫停於幀數: 1444
影片播放結束或讀取錯誤
第一個矩陣 (pause_points_1): [99, 221, 370, 526, 670, 801, 918, 1084, 1223, 1354]
第二個矩陣 (pause_points_2): [202, 332, 492, 635, 768, 896, 1051, 1194, 1329, 1444]


In [6]:
import os
path1 = "D:\\labdata\\MOCAP\\recordings_copy\\受試者11\\recording_20241204_113115\\"
skeleton_file_path = os.path.join(path1, 'yolo_skeleton_coordinates_1st_interp.txt')
bar_file_path1 = os.path.join(path1, 'yolo_coordinates_interpolated.txt')
video_path = os.path.join(path1, 'vision1.avi')
output_folder = os.path.join(path1, 'clips')
output_folder1 = os.path.join(path1, 'skeleton')
output_folder2 = os.path.join(path1, 'bar')
P1 = [99, 221, 370, 526, 670, 801, 918, 1084, 1223, 1354]
P2 = [202, 332, 492, 635, 768, 896, 1051, 1194, 1329, 1444]
split_video(video_path, output_folder, P1,P2,bar_file_path1, fps=30)
split_skeleton_data(skeleton_file_path, output_folder1,P1,P2,bar_file_path1)
split_bar_data(bar_file_path1, output_folder2, P1, P2,bar_file_path1)

影片儲存: D:\labdata\MOCAP\recordings_copy\受試者11\recording_20241204_113115\clips\clip_1.mp4（X 變化範圍: 349.27）
影片儲存: D:\labdata\MOCAP\recordings_copy\受試者11\recording_20241204_113115\clips\clip_2.mp4（X 變化範圍: 340.34）
影片儲存: D:\labdata\MOCAP\recordings_copy\受試者11\recording_20241204_113115\clips\clip_3.mp4（X 變化範圍: 332.60）
影片儲存: D:\labdata\MOCAP\recordings_copy\受試者11\recording_20241204_113115\clips\clip_4.mp4（X 變化範圍: 335.30）
影片儲存: D:\labdata\MOCAP\recordings_copy\受試者11\recording_20241204_113115\clips\clip_5.mp4（X 變化範圍: 341.03）
影片儲存: D:\labdata\MOCAP\recordings_copy\受試者11\recording_20241204_113115\clips\clip_6.mp4（X 變化範圍: 355.55）
影片儲存: D:\labdata\MOCAP\recordings_copy\受試者11\recording_20241204_113115\clips\clip_7.mp4（X 變化範圍: 337.14）
影片儲存: D:\labdata\MOCAP\recordings_copy\受試者11\recording_20241204_113115\clips\clip_8.mp4（X 變化範圍: 335.75）
影片儲存: D:\labdata\MOCAP\recordings_copy\受試者11\recording_20241204_113115\clips\clip_9.mp4（X 變化範圍: 330.25）
影片儲存: D:\labdata\MOCAP\recordings_copy\受試者11\recording_20241204_

In [4]:
##把angle.txt按2d bar的幀數來切割影像
import os
import pandas as pd

root_dir = "D:\\labdata\\MOCAP\\recordings_copy\\受試者19\\recording_20241206_113600"
output_folder_name = "angle_int"

for category in os.listdir(root_dir):
    category_path = os.path.join(root_dir, category)
    if not os.path.isdir(category_path):
        continue

    for recording in os.listdir(category_path):
        recording_path = os.path.join(category_path, recording)
        bar_path = os.path.join(root_dir, "bar")
        angle_path = os.path.join(root_dir, "yolo_skeleton_coordinates_vision1_interp.txt")
        output_path = os.path.join(root_dir, output_folder_name)

        if not os.path.exists(bar_path) or not os.path.isfile(angle_path):
            continue

        # 讀 angle.txt（用空格分隔）
        angle_df = pd.read_csv(angle_path, header=None, delim_whitespace=True)

        # 創建輸出資料夾
        os.makedirs(output_path, exist_ok=True)

        # 遍歷每個 bar/*.txt
        for file_name in sorted(os.listdir(bar_path)):
            if not file_name.endswith(".txt"):
                continue
            bar_file = os.path.join(bar_path, file_name)

            # 讀 bar 檔：取第一列與最後一列的第一個值（用逗號分隔）
            with open(bar_file, "r") as f:
                lines = f.readlines()
                if not lines:
                    continue
                start_frame = int(lines[0].strip().split(",")[0])
                end_frame = int(lines[-1].strip().split(",")[0])

            # 從 angle_df 中擷取對應幀數區間
            sub_angle_df = angle_df[(angle_df[0] >= start_frame) & (angle_df[0] <= end_frame)]

            # 存成新檔案（用逗號分隔）
            angle_output_name = f"angle_{os.path.splitext(file_name)[0].split('_')[-1]}.txt"
            sub_angle_df.to_csv(os.path.join(output_path, angle_output_name), index=False, header=False, sep=',')




TypeError: '>=' not supported between instances of 'str' and 'int'